In [1]:
!pip install -r requirements.txt

     |████████████████████████████████| 527 kB 3.2 MB/s eta 0:00:01
     |████████████████████████████████| 881.9 MB 33 kB/s s eta 0:00:01     |████████████▍                   | 342.0 MB 11.7 MB/s eta 0:00:46
     |████████████████████████████████| 409 kB 9.8 MB/s eta 0:00:01
     |████████████████████████████████| 133 kB 11.5 MB/s eta 0:00:01
     |████████████████████████████████| 840 kB 7.9 MB/s eta 0:00:01
     |████████████████████████████████| 603 kB 9.9 MB/s eta 0:00:01
     |████████████████████████████████| 5.9 MB 8.5 MB/s eta 0:00:01
     |████████████████████████████████| 178 kB 8.6 MB/s eta 0:00:01
     |████████████████████████████████| 155 kB 18.4 MB/s eta 0:00:01
     |████████████████████████████████| 77 kB 8.2 MB/s  eta 0:00:01
     |████████████████████████████████| 63 kB 3.5 MB/s  eta 0:00:01
     |████████████████████████████████| 155 kB 8.6 MB/s eta 0:00:01
     |████████████████████████████████| 151 kB 7.3 MB/s eta 0:00:01
     |████████████████████████████████| 4

     |████████████████████████████████| 270 kB 11.3 MB/s eta 0:00:01
     |████████████████████████████████| 952 kB 11.5 MB/s eta 0:00:01
  Created wheel for future: filename=future-0.18.3-py3-none-any.whl size=494240 sha256=9e239842e5319c2886a049bf28f164f5d46cf0520d5486dbd900b345a0d7c001
  Stored in directory: /root/.cache/pip/wheels/63/f1/0c/e56d12b3804345ce5ba34279cbfe583ecafdd1401551457330
  Created wheel for idna-ssl: filename=idna_ssl-1.1.0-py3-none-any.whl size=3949 sha256=33cb87ae187f76342dd2d499592158666241c32e4723333a87d010c933fb473b
  Stored in directory: /root/.cache/pip/wheels/6a/f5/9c/f8331a854f7a8739cf0e74c13854e4dd7b1af11b04fe1dde13
Successfully built future idna-ssl
  Attempting uninstall: setuptools
    Found existing installation: setuptools 51.1.1
    Uninstalling setuptools-51.1.1:
      Successfully uninstalled setuptools-51.1.1
  Attempting uninstall: tensorboard
    Found existing installation: tensorboard 1.15.0
    Uninstalling tensorboard-1.15.0:
      Succes

# GCN

## !python main.py --model_name TGCN \
--max_epochs 2 \
--learning_rate 0.001 \
--weight_decay 0 \
--batch_size 64 \
--hidden_dim 64 \
--settings supervised \
--loss "mse_with_regularizer" \
--data PEMS04

In [ ]:
"GRU", "LSTM", "GCN", "TGCN","TGCN_LSTM"

In [ ]:
from subprocess import Popen,PIPE
model = ["GRU"]
max_epochs = "100"
learning_rate = "0.001" 
weight_decay ="0"
enable_progress_bar = "False"
batch_size = "64"
hidden_dim = "64"
cell_dim = "64"
dropout = "0"
layer_2 = ["False"]
loss = ["mse_with_regularizer","mse_with_regularizer_l1","mse_with_regularizer_entropy","mse"]
data = ["PEMS08","PEMS04","losloop","shenzhen"]
gpus = "0"
for i in data:
    for j in loss:
        for k in model:
            for l in layer_2:
                print("Dataset:",i," Loss:",j," Model:",k,"Layer 2:",l)
                cmd_str = ["python",
                       "main.py",
                       "--enable_progress_bar",enable_progress_bar,
                       "--model_name",k,
                       "--max_epochs",max_epochs,
                       "--learning_rate",learning_rate,
                       "--weight_decay",weight_decay,
                       "--batch_size",batch_size,
                       "--hidden_dim",hidden_dim,
                       "--cell_dim",hidden_dim,
                       "--dropout",dropout,
                       "--loss",j,
                       "--layer_2",layer_2[1],
                       "--data",i,
                       "--gpus",gpus,
                      ]
                p = Popen(cmd_str,stdout=PIPE)
                for m in p.stdout:
                    print(m)

Dataset: PEMS08  Loss: mse_with_regularizer  Model: GRU


In [67]:
import time

# TGCN

In [68]:
%%time
!python main.py --model_name TGCN \
--max_epochs 5 \
--learning_rate 0.001 \
--weight_decay 0 \
--batch_size 32 \
--hidden_dim 64 \
--settings supervised \
--loss "mse_with_regularizer" \
--layer_2 False \
--dropout 0 \
--data losloop

[2023-04-07 11:35:20,305 INFO]{'logger': True, 'checkpoint_callback': None, 'enable_checkpointing': True, 'default_root_dir': None, 'gradient_clip_val': None, 'gradient_clip_algorithm': None, 'process_position': 0, 'num_nodes': 1, 'num_processes': 1, 'devices': None, 'gpus': None, 'auto_select_gpus': False, 'tpu_cores': None, 'ipus': None, 'log_gpu_memory': None, 'progress_bar_refresh_rate': None, 'enable_progress_bar': True, 'overfit_batches': 0.0, 'track_grad_norm': -1, 'check_val_every_n_epoch': 1, 'fast_dev_run': False, 'accumulate_grad_batches': None, 'max_epochs': 5, 'min_epochs': None, 'max_steps': -1, 'min_steps': None, 'max_time': None, 'limit_train_batches': 1.0, 'limit_val_batches': 1.0, 'limit_test_batches': 1.0, 'limit_predict_batches': 1.0, 'val_check_interval': 1.0, 'flush_logs_every_n_steps': None, 'log_every_n_steps': 50, 'accelerator': None, 'strategy': None, 'sync_batchnorm': False, 'precision': 32, 'enable_model_summary': True, 'weights_summary': 'top', 'weights_sav

In [ ]:
CPU times: user 3.22 s, sys: 794 ms, total: 4.01 s
Wall time: 4min 11s

# GRU

In [ ]:
!python main.py --model_name GRU \
--max_epochs 10 \
--learning_rate 0.001 \
--weight_decay 0 \
--batch_size 64 \
--hidden_dim 8 \
--settings supervised \
--data PEMS04

# LSTM

In [ ]:
!python main.py --model_name LSTM \
--max_epochs 1 \
--learning_rate 0.001 \
--weight_decay 0 \
--batch_size 64 \
--hidden_dim 8 \
--cell_dim 12 \
--settings supervised \
--data PEMS04

# TGCN-LSTM

In [ ]:
!python main.py --model_name TGCN_LSTM \
--max_epochs 20 \
--learning_rate 0.001 \
--weight_decay 0 \
--batch_size 64 \
--hidden_dim 8 \
--cell_dim 64 \
--settings supervised \
--data losloop

# SVD 

In [83]:
import pandas as pd
import numpy as np
import torch 
import time

def svd_low_rank_approximation(matrix, k):
    # Perform SVD on the matrix
    
    u, s, v = torch.svd(matrix)
    
    # Truncate the singular values and matrices to rank k
    s_k = s[:k]
    u_k = u[:, :k]
    v_k = v[:, :k]
    
    
    return (s_k,u_k,v_k)

def adjmat(adj):
    return torch.mm(torch.mm(adj[1], torch.diag(adj[0])), adj[2].t())

PEMS08_adj = pd.read_csv(r'data/pems08_adj.csv',header=None)
adj = np.array(PEMS08_adj)
adj = torch.tensor(adj)

data = np.load("data/pems08.npz")
data = data.f.data
df_data = pd.DataFrame(data[:,:,0])
df_data = df_data.values.tolist()
feat = np.array(df_data, dtype=np.float32)
feat = torch.tensor(feat)
print(feat.shape)

start = time.time()
for i in range(100):
    adjsvd= svd_low_rank_approximation(feat.t(), 10000)
    adjmatrix= adjmat(adjsvd)
end = time.time()
print(end-start)
start = time.time()
for i in range(100):
    adjsvd= svd_low_rank_approximation(feat.t(),17856)
    adjmatrix= adjmat(adjsvd)
end = time.time()
print(end-start)
print(adjmatrix.sum()**2/adj.sum()**2)
print("svd s u v = ",adjmatrix.shape)

torch.Size([17856, 170])
19.872730016708374
19.825145483016968
tensor(6.3904e+12, dtype=torch.float64)
svd s u v =  torch.Size([170, 17856])


In [90]:
import numpy as np
from numpy import linalg as la
np.random.seed(42)


def flip_signs(A, B):
    """
    utility function for resolving the sign ambiguity in SVD
    http://stats.stackexchange.com/q/34396/115202
    """
    signs = np.sign(A) * np.sign(B)
    return A, B * signs


# Let the data matrix X be of n x p size,
# where n is the number of samples and p is the number of variables
n, p = 10, 170
data = np.load("data/pems08.npz")
data = data.f.data
df_data = pd.DataFrame(data[:,:,0])
df_data = df_data.values.tolist()
X = np.array(df_data, dtype=np.float32)
X -= np.mean(X,axis=0)

# the p x p covariance matrix
C = np.cov(X, rowvar=False)
print("C = \n", C)
# C is a symmetric matrix and so it can be diagonalized:
l, principal_axes = la.eig(C)
# sort results wrt. eigenvalues
idx = l.argsort()[::-1]
l, principal_axes = l[idx], principal_axes[:, idx]
# the eigenvalues in decreasing order
print("l = \n", l)
# a matrix of eigenvectors (each column is an eigenvector)
print("V = \n", principal_axes)
# projections of X on the principal axes are called principal components
principal_components = X.dot(principal_axes)
print("Y = \n", principal_components)

# we now perform singular value decomposition of X
# "economy size" (or "thin") SVD
U, s, Vt = la.svd(X, full_matrices=False)
V = Vt.T
S = np.diag(s)

# 1) then columns of V are principal directions/axes.
assert np.allclose(*flip_signs(V, principal_axes))

# 2) columns of US are principal components
assert np.allclose(*flip_signs(U.dot(S), principal_components))

# 3) singular values are related to the eigenvalues of covariance matrix
assert np.allclose((s ** 2) / (n - 1), l)

# 8) dimensionality reduction
k = 1
PC_k = principal_components[:, 0:k]
US_k = U[:, 0:k].dot(S[0:k, 0:k])
print(PC_k, X[:,:])
assert np.allclose(*flip_signs(PC_k, US_k))

# 10) we used "economy size" (or "thin") SVD
assert U.shape == (n, p)
assert S.shape == (p, p)
assert V.shape == (p, p)

C = 
 [[21299.84727986 18898.13165904 12943.61253019 ... 13181.69580527
  17710.40767453  2033.24825426]
 [18898.13165904 23412.79781435 15126.26361152 ... 14584.91752615
  16745.71635712  2402.4195232 ]
 [12943.61253019 15126.26361152 12876.44453819 ...  9731.1222134
  11263.29372559  1705.03653584]
 ...
 [13181.69580527 14584.91752615  9731.1222134  ... 10660.86941215
  11847.14031209  1574.06524126]
 [17710.40767453 16745.71635712 11263.29372559 ... 11847.14031209
  17624.32361205  1766.11560189]
 [ 2033.24825426  2402.4195232   1705.03653584 ...  1574.06524126
   1766.11560189   331.49667177]]
l = 
 [ 1.72126341e+06+0.00000000e+00j  1.09668122e+05+0.00000000e+00j
  2.93000710e+04+0.00000000e+00j  2.03012635e+04+0.00000000e+00j
  1.93070108e+04+0.00000000e+00j  1.41062700e+04+0.00000000e+00j
  1.36642991e+04+0.00000000e+00j  1.19265608e+04+0.00000000e+00j
  1.07811271e+04+0.00000000e+00j  9.41204786e+03+0.00000000e+00j
  8.52796785e+03+0.00000000e+00j  7.67032353e+03+0.00000000e+00j

AssertionError: 